In [23]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
import torch
print(torch.__version__)
print(torch.cuda.is_available())

0.4.0
True


In [24]:
!pip3 install Pillow==4.0.0
!pip3 install PIL
!pip3 install image

    100% |████████████████████████████████| 5.6MB 7.3MB/s 
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
  Found existing installation: Pillow 5.4.0
    Uninstalling Pillow-5.4.0:
      Successfully uninstalled Pillow-5.4.0
  Could not find a version that satisfies the requirement PIL (from versions: )
No matching distribution found for PIL


In [0]:
import random
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import pandas as pd
import os
from PIL import Image

In [0]:
!wget https://cdn.skillenza.com/files/5ee0a2e1-bf6d-4173-8244-e75fa2d7bbe2/training.5k.zip
!unzip training.5k.zip  

In [0]:
NUM_EPOCHS = 10
LR = 1e-4
BATCH_SIZE = 4
GAMMA = 0.95
curr_lr = LR

In [0]:
tfms = transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor(), transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

In [0]:
categories = pd.read_csv("/content/training/solution.csv")["category"]
def getitem(idx):
    img_name = os.path.join("/content/training/training/", str(idx+1) + ".png")
    image = Image.open(img_name)
    if tfms:
        image = tfms(image)
    category = categories[idx]
    sample = {'image': image, 'category': category}

    return sample

In [0]:
a = np.arange(0,5000)
np.random.shuffle(a)
train_indexes = a[:4901]
test_indexes = a[4901:5001]

In [0]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

In [0]:
model = models.resnet34(pretrained=True)

model = nn.Sequential(*list(model.children())[:-2],
                     nn.Conv2d(512, 6, 3, 1),
                     nn.AdaptiveAvgPool2d(1), Flatten(),
                     nn.LogSoftmax())

if torch.cuda.is_available():
    model = model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9)

In [0]:
#print(model.children)
count = 0
for param in model.parameters():
    param.requires_grad = False
    count += 1
    if count >= 8:
        break

In [0]:
def adjust_learning_rate(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [0]:
losses = []
accuracies = []

In [0]:
def validation_accuracy(model):
  correct = 0
  
  with torch.no_grad():
    for idx in range(0,99):
      data = getitem(test_indexes[idx])
      image = data['image']
      exp = data['category']
      image = image.unsqueeze(0).cuda()
      target = model(image)
      ans = (torch.argmax(target).cpu().numpy()+1)
      if (ans == exp):
        correct +=1
  print(correct/99)

  

In [0]:
for epoch in range(NUM_EPOCHS):
    total_loss = 0
    accuracy = 0
    count = 0

    adjust_learning_rate(optimizer, curr_lr)
    curr_lr *= GAMMA
    
    optimizer.zero_grad()
    for c in range(0,int(4900/BATCH_SIZE)):
        optimizer.zero_grad()
        images = torch.zeros([4,3,224,224])
        targets = []
        for p in range(0,4):
            data = getitem(train_indexes[4*c+p])
            images[p,:,:,:] = data['image']
            targets.append(np.asscalar(data["category"])-1)
        targets = torch.LongTensor(targets)
        if torch.cuda.is_available():
            images = images.cuda()
            targets = targets.cuda()
        
        outs = model(images)
        loss = criterion(outs, targets)
        loss.backward()
        optimizer.step()
                
        total_loss += loss.data
        accuracy += torch.sum(torch.argmax(outs, dim=1) == targets)
        count += 1

        if c%300 == 0:
            print("Epoch:", epoch, "Iter:", c, "average batch loss:", total_loss/(c+1))
    
    
    print("\nEpoch:", epoch, "Total loss:", total_loss)
    acc = 0.25*accuracy.cpu().numpy()/count
    print("Epoch:", epoch, "Accuracy:", acc,'\n') 
    print('Validation accuracy:',validation_accuracy(model))
    losses.append(total_loss)
    accuracies.append(acc)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:91: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch: 0 Iter: 0 average batch loss: tensor(1.9991, device='cuda:0')
Epoch: 0 Iter: 300 average batch loss: tensor(1.8495, device='cuda:0')
Epoch: 0 Iter: 600 average batch loss: tensor(1.6341, device='cuda:0')
Epoch: 0 Iter: 900 average batch loss: tensor(1.3926, device='cuda:0')
Epoch: 0 Iter: 1200 average batch loss: tensor(1.1956, device='cuda:0')

Epoch: 0 Total loss: tensor(1444.3320, device='cuda:0')
Epoch: 0 Accuracy: 0.5477551020408163 

0.6464646464646465
Validation accuracy: None
Epoch: 1 Iter: 0 average batch loss: tensor(1.00000e-02 *
       2.0389, device='cuda:0')
Epoch: 1 Iter: 300 average batch loss: tensor(0.4543, device='cuda:0')
Epoch: 1 Iter: 600 average batch loss: tensor(0.3679, device='cuda:0')
Epoch: 1 Iter: 900 average batch loss: tensor(0.3064, device='cuda:0')
Epoch: 1 Iter: 1200 average batch loss: tensor(0.2560, device='cuda:0')

Epoch: 1 Total loss: tensor(309.9279, device='cuda:0')
Epoch: 1 Accuracy: 0.9193877551020408 

0.8686868686868687
Validation acc

In [0]:
torch.save(model.state_dict(), "resnet34_model1.pt")

In [0]:
model.load_state_dict(torch.load("resnet34_model1.pt"))
#model.eval()

In [133]:
validation_accuracy(model)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:91: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


0.7878787878787878
